# Metamodeling - GBRT

Objective: Develop and Optimize GBRT to predict oil spill

Outline of this script:

Data Preparation
Develop SVR model
Fit and measure performance
Apply Grid Search, Random Search, HyperOpt and TPOT
Compare their performance
Input for this script: A dataset which include

Output of this script: A few dataframe that contains performance measures (e.g. R^2 and RMSE) and design of the algorithm

Date: Feb 20, 2021

Written by: Tanmoy Das

## Import libraries & dataset

In [1]:
# Import libraries
import time
import pandas as pd
import numpy as np

In [2]:
# ML modeling
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [4]:
# Import Training dataset  
X_train_dev = pd.read_csv('Dataset for metamodeling/X_train_dev.csv', header=0).copy()
X_val = pd.read_csv('Dataset for metamodeling/X_val.csv', header=0).copy()
y_train_dev = pd.read_csv('Dataset for metamodeling/y_train_dev.csv', header=0).copy()
y_val = pd.read_csv('Dataset for metamodeling/y_val.csv', header=0).copy()

In [5]:
display(X_train_dev.describe().T)

,count,mean,std,min,25%,50%,75%,max
$type_A$,74880.0,0.490008,0.334118,0.0,0.166667,0.500000,0.833333,1.0
$type_B$,74880.0,0.491090,0.331719,0.0,0.230769,0.538462,0.769231,1.0
$type_{oil}$,74880.0,0.492446,0.376486,0.0,0.000000,0.333333,1.000000,1.0
$velocity_A$,74880.0,0.498944,0.288640,0.0,0.247730,0.498735,0.749022,1.0
$velocity_B$,74880.0,0.499480,0.288556,0.0,0.249848,0.499230,0.750018,1.0
$angle_{collision}$,74880.0,0.500327,0.288654,0.0,0.250539,0.500202,0.749741,1.0
$location_{impact.B}$,74880.0,0.398842,0.242892,0.0,0.197668,0.361345,0.575490,1.0
$displacement_A$,74880.0,0.459782,0.325586,0.0,0.122403,0.457617,0.673070,1.0
$displacement_B$,74880.0,0.405861,0.288774,0.0,0.219923,0.362168,0.543024,1.0
$length_B$,74880.0,0.639653,0.318023,0.0,0.442238,0.748682,0.937961,1.0


In [5]:
model_gbrt = GradientBoostingRegressor()
model_gbrt.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

## Initial Model

In [5]:
model = GradientBoostingRegressor(max_depth = 4, learning_rate=1, n_estimators=10, random_state=0)
model_gbrt = MultiOutputRegressor(model, n_jobs=30) #

model_gbrt.fit(X_train_dev, y_train_dev)
print(model_gbrt.score(X_val, y_val))

0.7937048131362022


# Random Search

In [18]:
# 04.04.2021 Tanmoy at 10.45am
model = MultiOutputRegressor(GradientBoostingRegressor())

hyperparameters = dict(estimator__learning_rate= np.linspace(0.01,.8,10),#[0.001, .05, 0.01, 0.1, 0.2, 0.5, 0.9], 
                    estimator__n_estimators=[2,3,4,5,10,20],
                 #    estimator__criterion=['friedman_mse', 'mse'], estimator__min_samples_split=[2, 4, 7, 10],
                     estimator__max_depth=[3, 5, 10, 15, 20] ) #, estimator__min_samples_leaf=[1, 2, 3, 5, 8, 10],
                #     estimator__min_impurity_decrease=[0, 0.2, 0.4, 0.6, 0.8],
                 #    estimator__max_leaf_nodes=[5, 10, 20, 30, 50, 100, 300])
number_models = 10

randomized_search_model_gbrt = RandomizedSearchCV(model, hyperparameters, #n_iter=number_models,  #random_state=0, 
                                                  scoring=['r2', 'neg_mean_squared_error', 'neg_median_absolute_error'],
                                                   n_jobs=50, refit=False, cv=2, verbose=0, #n_jobs=1 to get Verbose output (runtime of each model) for each iteration
                                                   pre_dispatch='2*n_jobs', error_score='raise', return_train_score=True)

randomized_search_model_gbrt = randomized_search_model_gbrt.fit(X_train_dev, y_train_dev)
#print('Best Parameters = {}'.format(randomized_search_model_gbrt.best_params_))
# tuned_model = randomized_search_model_gbrt.best_estimator_

# https://stackoverflow.com/questions/43532811/gridsearch-over-multioutputregressor/52562463#52562463


# Calculate scores on validation training set +++ 
learning_rate = [item['estimator__learning_rate'] for item in randomized_search_model_gbrt.cv_results_['params']]
n_estimators = [item['estimator__n_estimators'] for item in randomized_search_model_gbrt.cv_results_['params']]
max_depth = [item['estimator__max_depth'] for item in randomized_search_model_gbrt.cv_results_['params']]

r2_gbrt = list(randomized_search_model_gbrt.cv_results_['mean_test_r2'])
neg_mean_squared_error_gbrt = list(randomized_search_model_gbrt.cv_results_['mean_test_neg_mean_squared_error'])
neg_median_absolute_error_gbrt = list(randomized_search_model_gbrt.cv_results_['mean_test_neg_median_absolute_error'])


df_rands_gbrt_perfm = pd.DataFrame([learning_rate, n_estimators, max_depth, 
                                    r2_gbrt, neg_mean_squared_error_gbrt, neg_median_absolute_error_gbrt]).T
df_rands_gbrt_perfm.columns = ['learning_rate','n_estimators', 'max_depth', 
                               'r2_gbrt', 'neg_mean_squared_error_gbrt','neg_median_absolute_error_gbrt']
# df_rands_dnn_perfm.groupby(['Learning Rate']).mean()
#display(df_rands_gbrt_perfm)
display(df_rands_gbrt_perfm.sort_values(by=['r2_gbrt'],ascending=False)) #ascending=False

,learning_rate,n_estimators,max_depth,r2_gbrt,neg_mean_squared_error_gbrt,neg_median_absolute_error_gbrt
7,0.448889,5.0,10.0,0.908323,-0.003641,-0.015363
1,0.448889,4.0,20.0,0.872836,-0.005085,-0.017101
6,0.800000,5.0,20.0,0.868438,-0.005151,-0.002933
5,0.800000,3.0,20.0,0.868254,-0.005163,-0.003951
0,0.361111,4.0,20.0,0.861596,-0.005677,-0.030874
8,0.185556,5.0,20.0,0.782763,-0.009656,-0.065057
4,0.536667,4.0,5.0,0.779106,-0.009906,-0.042250
2,0.273333,3.0,10.0,0.745657,-0.011359,-0.070110
3,0.097778,5.0,5.0,0.421054,-0.027082,-0.114142
9,0.097778,2.0,3.0,0.147944,-0.040693,-0.153109


# TPOT

In [10]:
# TPOT
from tpot import TPOTRegressor

config_dict = { 'sklearn.ensemble.GradientBoostingRegressor':  { 'learning_rate' : np.linspace(0.01,.9,100),
                    'max_depth' : list(range(2, 11, 4)) }}
tpot_gbrt_model = TPOTRegressor(generations=2, population_size=5,
                    verbosity=2, offspring_size=2,
                    scoring='r2', cv=2, config_dict = config_dict)

In [11]:
from sklearn.multioutput import MultiOutputRegressor
TPOT_multi = MultiOutputRegressor(tpot_gbrt_model)
TPOT_multi.fit(X_train_dev, y_train_dev)

print('TPOT EA on GBRT: ')
print(TPOT_multi.score(X_val, y_val))


Generation 1 - Current best internal CV score: 0.8660274667468869

Generation 2 - Current best internal CV score: 0.8912501809554101

Best pipeline: GradientBoostingRegressor(GradientBoostingRegressor(input_matrix, learning_rate=0.12686868686868688, max_depth=10), learning_rate=0.882020202020202, max_depth=2)



Generation 1 - Current best internal CV score: 0.9043795125872096

Generation 2 - Current best internal CV score: 0.9043795125872096

Best pipeline: GradientBoostingRegressor(GradientBoostingRegressor(input_matrix, learning_rate=0.15383838383838386, max_depth=10), learning_rate=0.27070707070707073, max_depth=10)



Generation 1 - Current best internal CV score: 0.9025658158837089

Generation 2 - Current best internal CV score: 0.9025658158837089

Best pipeline: GradientBoostingRegressor(CombineDFs(input_matrix, input_matrix), learning_rate=0.2976767676767677, max_depth=10)



Generation 1 - Current best internal CV score: 0.9541953258740021

Generation 2 - Current best internal CV score: 0.9541953258740021

Best pipeline: GradientBoostingRegressor(input_matrix, learning_rate=0.17181818181818184, max_depth=10)



Generation 1 - Current best internal CV score: 0.9668641287693318

Generation 2 - Current best internal CV score: 0.9668641287693318

Best pipeline: GradientBoostingRegressor(input_matrix, learning_rate=0.2167676767676768, max_depth=10)



Generation 1 - Current best internal CV score: 0.9631859774944118

Generation 2 - Current best internal CV score: 0.9631859774944118

Best pipeline: GradientBoostingRegressor(GradientBoostingRegressor(input_matrix, learning_rate=0.3156565656565657, max_depth=10), learning_rate=0.7741414141414142, max_depth=10)



Generation 1 - Current best internal CV score: 0.9651365741933935

Generation 2 - Current best internal CV score: 0.9651365741933935

Best pipeline: GradientBoostingRegressor(GradientBoostingRegressor(input_matrix, learning_rate=0.5224242424242425, max_depth=10), learning_rate=0.4325252525252526, max_depth=6)
TPOT EA on GBRT: 
0.9446493010904197


# Grid Search

In [19]:
# 05.01.2021 Tanmoy at 10.45am
model = MultiOutputRegressor(GradientBoostingRegressor())

hyperparameters = dict(estimator__learning_rate=[0.001, 0.1, 0.2, 0.5, 0.9], 
                    estimator__n_estimators=[1,3,4,5,10],
                    estimator__max_depth=[3, 5, 10] ) 


grid_search_model_gbrt = GridSearchCV(model, hyperparameters, #n_iter=number_models,  #random_state=0, 
                                                  scoring=['r2', 'neg_mean_squared_error', 'neg_median_absolute_error'],
                                                   n_jobs=50, refit=False, cv=2, verbose=0, #n_jobs=1 to get Verbose output (runtime of each model) for each iteration
                                                   pre_dispatch='2*n_jobs', error_score='raise', return_train_score=True)

grid_search_model_gbrt = grid_search_model_gbrt.fit(X_train_dev, y_train_dev)

learning_rate = [item['estimator__learning_rate'] for item in grid_search_model_gbrt.cv_results_['params']]
n_estimators = [item['estimator__n_estimators'] for item in grid_search_model_gbrt.cv_results_['params']]
max_depth = [item['estimator__max_depth'] for item in grid_search_model_gbrt.cv_results_['params']]

r2_gbrt = list(grid_search_model_gbrt.cv_results_['mean_test_r2'])
neg_mean_squared_error_gbrt = list(grid_search_model_gbrt.cv_results_['mean_test_neg_mean_squared_error'])
neg_median_absolute_error_gbrt = list(grid_search_model_gbrt.cv_results_['mean_test_neg_median_absolute_error'])


df_grids_gbrt_perfm = pd.DataFrame([learning_rate, n_estimators, max_depth, 
                                    r2_gbrt, neg_mean_squared_error_gbrt, neg_median_absolute_error_gbrt]).T
df_grids_gbrt_perfm.columns = ['learning_rate','n_estimators', 'max_depth', 
                               'r2_gbrt', 'neg_mean_squared_error_gbrt','neg_median_absolute_error_gbrt']
# df_rands_dnn_perfm.groupby(['Learning Rate']).mean()
display(df_grids_gbrt_perfm.sort_values(by=['r2_gbrt'],ascending=False)) #ascending=False

,learning_rate,n_estimators,max_depth,r2_gbrt,neg_mean_squared_error_gbrt,neg_median_absolute_error_gbrt
59,0.500,10.0,10.0,0.919382,-0.003071,-0.013188
58,0.500,5.0,10.0,0.910391,-0.003535,-0.013929
44,0.200,10.0,10.0,0.901756,-0.003964,-0.023332
57,0.500,4.0,10.0,0.900875,-0.003964,-0.017037
56,0.500,3.0,10.0,0.877629,-0.004985,-0.026289
...,...,...,...,...,...,...
2,0.001,4.0,3.0,0.003463,-0.048097,-0.176225
1,0.001,3.0,3.0,0.002596,-0.048141,-0.176329
10,0.001,1.0,10.0,0.001689,-0.048190,-0.176490
5,0.001,1.0,5.0,0.001247,-0.048211,-0.176516


In [23]:
df_rands_gbrt_perfm.to_csv('df_rands_gbrt_perfm.csv')

# Bayesian Optimization

In [20]:
model_gbrt = GradientBoostingRegressor()
model_gbrt.get_params()
import numpy as np

In [21]:
from sklearn.model_selection import cross_val_score
from hyperopt import hp
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
# Setting the space
space = {
    'max_depth': hp.uniform('max_depth', 4, 20),
    'learning_rate': hp.uniform('learning_rate', 0.001, .1),
   'n_estimators': hp.uniform('n_estimators', 1,20)
}# r2_score
def objective_r2(params_bo):
    params_bo = {'max_depth': int(params_bo['max_depth']),
        'learning_rate': params_bo['learning_rate'],
               'n_estimators': int(params_bo['n_estimators'])}
    gbrt_bo = GradientBoostingRegressor(**params_bo)
    #
    gbrt_bo_multi = MultiOutputRegressor(gbrt_bo)
    gbrt_bo_multi.fit(X_train_dev, y_train_dev)
    #r2_bo = (-1)* cross_val_score(neural_net_bo, X_train, y_train, scoring='r2', cv=5).mean()
    y_pred_bo = gbrt_bo_multi.predict(X_val)
    r2_bo = r2_score(y_val, y_pred_bo) # If False, returns RMSE value
    print('r2_bo: ', r2_bo)
    print('max_depth, learning_rate, n_estimators', 
          params_bo['max_depth'], params_bo['learning_rate'], params_bo['n_estimators'])
    return r2_bo

In [22]:
from hyperopt import fmin, tpe
best_result = fmin(fn=objective_r2,
            space=space,
            max_evals=10,
            rstate=np.random.RandomState(22),
            algo=tpe.suggest)
# print the best parameter

r2_bo:                                                                                                                 
0.26298845580744457                                                                                                    
max_depth, learning_rate, n_estimators                                                                                 
4                                                                                                                      
0.016667871589894637                                                                                                   
19                                                                                                                     
r2_bo:                                                                                                                 
0.6823178130601081                                                                                                     
max_depth, learning_rate, n_estimators  